# Tutorial 8: Capstone — The Guild Economy

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/buildLittleWorlds/blockchain-tokens-and-incentives/blob/main/notebooks/tutorial_08_capstone_the_guild_economy.ipynb)

---

> *"A rule says 'don't steal.' An incentive makes stealing cost more than it gains. The first requires a judge. The second requires only arithmetic."*
>
> — Brenn Auster, *Self-Enforcing Agreements* (Year 867)

---

## The Complete System

This capstone brings together everything from Tutorials 01-07 into a single, running economic simulation:

| Layer | Component | Tutorial | Role |
|-------|-----------|----------|------|
| Foundation | Prisoner's Dilemma | 01 | Why defection dominates |
| Theory | Game Theory | 02 | When cooperation can survive |
| Design | Mechanism Design | 03 | Engineering good equilibria |
| Currency | ERC-20 Tokens | 04 | Programmable money |
| Enforcement | Staking/Slashing | 05 | Making honesty dominant |
| Exchange | AMM | 06 | Trustless commodity trading |
| Risk | MEV | 07 | Extraction and mitigation |
| **Integration** | **Guild Economy** | **08** | **Everything running together** |

We will build a complete token economy with 30 guild agents running for 1,000 rounds. Agents earn membership tokens, stake for reputation, trade through AMMs, and face MEV extraction. The system measures itself: Gini inequality, collusion risk, participation rates, and overall health.

This is Auster's vision fully realized — a self-enforcing economy where the mathematics make honesty the cheapest strategy.

## Learning Objectives

In this tutorial, you will:

1. **Build the token economy** — ERC-20 membership tokens with staked reputation
2. **Integrate the AMM** — automated commodity exchange with fee distribution
3. **Run a 1,000-round simulation** — 30 agents with different strategies
4. **Measure system health** — Gini coefficient, collusion risk, participation, MEV burden
5. **Analyze emergent behavior** — what happens when incentives, staking, and trading interact

**Technical Concepts:**
- Agent-based economic simulation
- Token ecosystem integration (currency + staking + AMM)
- System health metrics and economic dashboards
- Emergent behavior from mechanism design

## Setup

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from collections import defaultdict

BASE_URL = "https://raw.githubusercontent.com/buildLittleWorlds/densworld-datasets/main/data/"

# Load all course datasets
guild_trades = pd.read_csv(BASE_URL + "guild_trade_ledger.csv")
game_theory = pd.read_csv(BASE_URL + "game_theory_outcomes.csv")
staking = pd.read_csv(BASE_URL + "staking_rewards.csv")
amm_pools = pd.read_csv(BASE_URL + "amm_liquidity_pools.csv")
mev_log = pd.read_csv(BASE_URL + "mev_extraction_log.csv")
scholars = pd.read_csv(BASE_URL + "scholars.csv")

print("Datasets loaded:")
for name, df in [('guild_trades', guild_trades), ('game_theory', game_theory),
                  ('staking', staking), ('amm_pools', amm_pools),
                  ('mev_log', mev_log), ('scholars', scholars)]:
    print(f"  {name:15s} {len(df):5d} rows")

## Part 1: The Economy Architecture

The guild economy has four layers, each built from a previous tutorial:

1. **GuildCoin (GLD)** — ERC-20 membership token (Tutorial 04)
2. **Reputation (REP)** — Non-transferable staked reputation (Tutorial 05)
3. **Commodity Pools** — AMM exchange for guild commodities (Tutorial 06)
4. **Block Builder** — Transaction ordering with MEV extraction (Tutorial 07)

In [ ]:
class GuildToken:
    """Simplified ERC-20 for the simulation."""
    def __init__(self, name, symbol):
        self.name = name
        self.symbol = symbol
        self.balances = defaultdict(float)
        self.total_supply = 0
    
    def mint(self, to, amount):
        self.balances[to] += amount
        self.total_supply += amount
    
    def burn(self, holder, amount):
        self.balances[holder] = max(0, self.balances[holder] - amount)
        self.total_supply = max(0, self.total_supply - amount)
    
    def transfer(self, sender, to, amount):
        if self.balances[sender] >= amount:
            self.balances[sender] -= amount
            self.balances[to] += amount
            return True
        return False

class ReputationSystem:
    """Non-transferable staked reputation with slashing."""
    LOCKUP_MULTIPLIERS = {1: 1.0, 2: 1.5, 4: 2.5, 8: 4.0}
    BASE_REWARD = 0.03  # 3% per round
    SLASH_RATE = 0.10   # 10% of stake for violations
    MAX_VIOLATIONS = 3
    
    def __init__(self):
        self.stakes = {}  # agent → {amount, lockup, violations}
    
    def stake(self, agent, amount, lockup):
        self.stakes[agent] = {'amount': amount, 'lockup': lockup, 'violations': 0}
    
    def reward(self, agent):
        if agent not in self.stakes:
            return 0
        s = self.stakes[agent]
        mult = self.LOCKUP_MULTIPLIERS.get(s['lockup'], 1.0)
        return s['amount'] * self.BASE_REWARD * mult
    
    def slash(self, agent):
        if agent not in self.stakes:
            return 0
        s = self.stakes[agent]
        penalty = s['amount'] * self.SLASH_RATE
        s['amount'] -= penalty
        s['violations'] += 1
        return penalty
    
    def is_ejected(self, agent):
        if agent not in self.stakes:
            return True
        return self.stakes[agent]['violations'] >= self.MAX_VIOLATIONS

class CommodityPool:
    """Simplified AMM for guild commodity trading."""
    FEE = 0.003
    
    def __init__(self, token_a, token_b, reserve_a, reserve_b):
        self.token_a = token_a
        self.token_b = token_b
        self.reserve_a = reserve_a
        self.reserve_b = reserve_b
        self.total_fees = 0
        self.trade_count = 0
    
    @property
    def price(self):
        return self.reserve_b / self.reserve_a if self.reserve_a > 0 else 0
    
    @property
    def tvl(self):
        return self.reserve_a + self.reserve_b
    
    def swap(self, amount_in, direction='a_to_b'):
        fee = amount_in * self.FEE
        net_in = amount_in - fee
        self.total_fees += fee
        self.trade_count += 1
        
        k = self.reserve_a * self.reserve_b
        if direction == 'a_to_b':
            new_a = self.reserve_a + net_in
            new_b = k / new_a
            out = self.reserve_b - new_b
            self.reserve_a = new_a + fee
            self.reserve_b = new_b
        else:
            new_b = self.reserve_b + net_in
            new_a = k / new_b
            out = self.reserve_a - new_a
            self.reserve_b = new_b + fee
            self.reserve_a = new_a
        return out

print("Economy components initialized:")
print("  GuildToken — ERC-20 membership currency")
print("  ReputationSystem — Staked reputation with slashing")
print("  CommodityPool — AMM for trustless commodity exchange")

## Part 2: The Agent Model

Each guild agent has:
- A **strategy** (honest, opportunistic, or predatory)
- A **GLD balance** (spending money)
- A **staked amount** (reputation)
- A **commodity inventory** (for trading)

Each round, agents decide: trade honestly, attempt fraud, provide liquidity, or attempt MEV extraction.

In [ ]:
class GuildAgent:
    """An agent in the guild economy."""
    
    def __init__(self, name, strategy, initial_gld, lockup):
        self.name = name
        self.strategy = strategy  # 'honest', 'opportunistic', 'predatory'
        self.gld = initial_gld
        self.lockup = lockup
        self.commodities = {'iron': 50.0, 'salt': 80.0}
        self.trade_history = []
        self.rounds_active = 0
        self.total_profit = 0
        self.mev_attempts = 0
        self.mev_profits = 0
        self.times_slashed = 0
    
    def decide_action(self, pool_price, detection_prob):
        """Decide what to do this round based on strategy."""
        if self.strategy == 'honest':
            return 'trade_honest'
        elif self.strategy == 'opportunistic':
            # Cheat when detection probability is low
            if np.random.random() > detection_prob:
                return np.random.choice(['trade_honest', 'attempt_fraud'], p=[0.7, 0.3])
            return 'trade_honest'
        else:  # predatory
            r = np.random.random()
            if r < 0.4:
                return 'attempt_fraud'
            elif r < 0.6:
                return 'attempt_mev'
            return 'trade_honest'

# Create the agent population
np.random.seed(867)

# Use real Densworld names
agent_names = [
    'Brenn Auster', 'Torren Gael', 'Tessyn Mord', 'Kellis Vorn', 'Mollen Vek',
    'Serath Kyne', 'Vagabu Olt', 'Dessa Morin', 'Pemlik Tross', 'Hask Berrol',
    'Boffa Trent', 'Yasho Krent', 'Grigsu Haldo', 'Rellen Voss', 'Sareth Moll',
    'Erris Pol', 'Pell Anster', 'Archon Vells', 'Kella Vorn', 'Vornis Keth',
    'Mira Strand', 'Tobin Creel', 'Danner Volk', 'Lessi Quorn', 'Marten Friis',
    'Okka Brevins', 'Tully Gesh', 'Winnow Hasp', 'Jerald Spane', 'Corra Delm'
]

# Strategy distribution: 60% honest, 25% opportunistic, 15% predatory
strategies = (
    ['honest'] * 18 +
    ['opportunistic'] * 7 +
    ['predatory'] * 5
)
np.random.shuffle(strategies)

agents = []
for name, strategy in zip(agent_names, strategies):
    initial_gld = np.random.uniform(500, 2000)
    lockup = np.random.choice([1, 2, 4, 8], p=[0.3, 0.3, 0.25, 0.15])
    agents.append(GuildAgent(name, strategy, initial_gld, lockup))

print(f"Guild Economy: {len(agents)} agents")
print(f"\nStrategy distribution:")
for s in ['honest', 'opportunistic', 'predatory']:
    count = sum(1 for a in agents if a.strategy == s)
    print(f"  {s:15s} {count:3d} agents ({count/len(agents):.0%})")

print(f"\nLockup distribution:")
for l in [1, 2, 4, 8]:
    count = sum(1 for a in agents if a.lockup == l)
    print(f"  {l}-year:  {count:3d} agents")

## Part 3: The Simulation Engine

Each round of the simulation:
1. Agents decide their action
2. Honest trades execute through the AMM
3. Fraud attempts are detected based on stake-weighted probability
4. MEV extraction is attempted by predatory agents
5. Staking rewards/slashing are processed
6. System health metrics are recorded

In [ ]:
class GuildEconomy:
    """The complete guild economy simulation."""
    
    def __init__(self, agents):
        self.agents = {a.name: a for a in agents}
        self.token = GuildToken('GuildCoin', 'GLD')
        self.reputation = ReputationSystem()
        self.pool = CommodityPool('iron', 'salt', 1500, 2400)
        self.round = 0
        self.history = []
        self.health_history = []
        self.mev_events = []
        
        # Initialize tokens and stakes
        for agent in agents:
            self.token.mint(agent.name, agent.gld)
            stake_amount = agent.gld * 0.3  # Stake 30% of holdings
            self.reputation.stake(agent.name, stake_amount, agent.lockup)
    
    def _detection_probability(self, agent_name):
        """Higher stake = higher detection probability (skin in the game)."""
        if agent_name not in self.reputation.stakes:
            return 0.2  # Low baseline for non-stakers
        stake = self.reputation.stakes[agent_name]['amount']
        # Detection scales with network-wide staking
        total_staked = sum(s['amount'] for s in self.reputation.stakes.values())
        return min(0.3 + stake / max(total_staked, 1) * 2, 0.9)
    
    def run_round(self):
        """Execute one round of the economy."""
        self.round += 1
        round_events = []
        
        active_agents = [a for a in self.agents.values() 
                         if not self.reputation.is_ejected(a.name)]
        
        for agent in active_agents:
            agent.rounds_active += 1
            detection_prob = self._detection_probability(agent.name)
            action = agent.decide_action(self.pool.price, detection_prob)
            
            if action == 'trade_honest':
                self._execute_honest_trade(agent, round_events)
            elif action == 'attempt_fraud':
                self._execute_fraud_attempt(agent, detection_prob, round_events)
            elif action == 'attempt_mev':
                self._execute_mev_attempt(agent, active_agents, round_events)
            
            # Staking rewards for non-slashed agents
            if not self.reputation.is_ejected(agent.name):
                reward = self.reputation.reward(agent.name)
                self.token.mint(agent.name, reward)
                agent.gld = self.token.balances[agent.name]
        
        # Record system health
        self._record_health()
        self.history.extend(round_events)
    
    def _execute_honest_trade(self, agent, events):
        """Execute an honest commodity trade through the AMM."""
        if np.random.random() < 0.5 and agent.commodities['iron'] > 5:
            amount = min(np.random.uniform(2, 15), agent.commodities['iron'])
            received = self.pool.swap(amount, 'a_to_b')
            agent.commodities['iron'] -= amount
            agent.commodities['salt'] += received
            agent.total_profit += received - amount * self.pool.price
        elif agent.commodities['salt'] > 5:
            amount = min(np.random.uniform(2, 20), agent.commodities['salt'])
            received = self.pool.swap(amount, 'b_to_a')
            agent.commodities['salt'] -= amount
            agent.commodities['iron'] += received
        
        events.append({'round': self.round, 'agent': agent.name, 'action': 'honest_trade'})
    
    def _execute_fraud_attempt(self, agent, detection_prob, events):
        """Attempt fraud — may be detected and slashed."""
        fraud_gain = np.random.uniform(10, 50)
        
        if np.random.random() < detection_prob:
            # Detected — slash!
            penalty = self.reputation.slash(agent.name)
            agent.times_slashed += 1
            agent.total_profit -= penalty
            events.append({'round': self.round, 'agent': agent.name, 
                           'action': 'fraud_detected', 'penalty': penalty})
        else:
            # Undetected — profit
            self.token.mint(agent.name, fraud_gain)
            agent.total_profit += fraud_gain
            events.append({'round': self.round, 'agent': agent.name,
                           'action': 'fraud_undetected', 'gain': fraud_gain})
    
    def _execute_mev_attempt(self, agent, all_agents, events):
        """Attempt MEV extraction on a random trader."""
        agent.mev_attempts += 1
        targets = [a for a in all_agents if a.name != agent.name]
        if not targets:
            return
        
        victim = np.random.choice(targets)
        # MEV success depends on trade volume and pool depth
        extraction = np.random.uniform(0.5, 5.0)
        gas_cost = np.random.uniform(1, 3)
        net = extraction - gas_cost
        
        if net > 0 and np.random.random() < 0.6:  # 60% success rate
            self.token.mint(agent.name, net)
            agent.mev_profits += net
            agent.total_profit += net
            self.mev_events.append({
                'round': self.round, 'extractor': agent.name, 
                'victim': victim.name, 'profit': net
            })
            events.append({'round': self.round, 'agent': agent.name,
                           'action': 'mev_success', 'profit': net})
        else:
            # Failed — pay gas anyway
            agent.total_profit -= gas_cost
            events.append({'round': self.round, 'agent': agent.name,
                           'action': 'mev_failed', 'cost': gas_cost})
    
    def _record_health(self):
        """Record system health metrics."""
        balances = np.array([self.token.balances[a.name] for a in self.agents.values()])
        balances = balances[balances > 0]
        
        # Gini coefficient
        if len(balances) > 1:
            sorted_b = np.sort(balances)
            n = len(sorted_b)
            gini = (2 * np.sum((np.arange(1, n+1) * sorted_b))) / (n * np.sum(sorted_b)) - (n + 1) / n
        else:
            gini = 0
        
        active = sum(1 for a in self.agents.values() 
                     if not self.reputation.is_ejected(a.name))
        ejected = len(self.agents) - active
        
        # Collusion risk: fraction of predatory agents still active
        active_predatory = sum(1 for a in self.agents.values()
                               if a.strategy == 'predatory' and 
                               not self.reputation.is_ejected(a.name))
        collusion_risk = active_predatory / max(active, 1)
        
        self.health_history.append({
            'round': self.round,
            'gini': gini,
            'active_agents': active,
            'ejected': ejected,
            'participation_rate': active / len(self.agents),
            'collusion_risk': collusion_risk,
            'pool_tvl': self.pool.tvl,
            'pool_price': self.pool.price,
            'total_supply': self.token.total_supply,
            'total_fees': self.pool.total_fees,
            'total_trades': self.pool.trade_count,
            'total_mev': len(self.mev_events)
        })

print("Guild Economy engine ready.")
print(f"  30 agents, 4 layers (token, reputation, AMM, MEV)")
print(f"  Ready to run 1,000 rounds")

## Part 4: Running the Simulation

In [ ]:
# Run 1,000 rounds
np.random.seed(867)
economy = GuildEconomy(agents)

N_ROUNDS = 1000
checkpoints = [100, 250, 500, 750, 1000]

print(f"Running {N_ROUNDS} rounds...\n")
for r in range(1, N_ROUNDS + 1):
    economy.run_round()
    
    if r in checkpoints:
        h = economy.health_history[-1]
        print(f"Round {r:5d}: active={h['active_agents']}, "
              f"gini={h['gini']:.3f}, collusion={h['collusion_risk']:.2f}, "
              f"trades={h['total_trades']}, TVL={h['pool_tvl']:.0f}")

print(f"\nSimulation complete: {N_ROUNDS} rounds, {len(economy.history)} events")

## Part 5: System Health Dashboard

In [ ]:
health = pd.DataFrame(economy.health_history)

fig, axes = plt.subplots(3, 2, figsize=(14, 14))

# Gini coefficient over time
axes[0, 0].plot(health['round'], health['gini'], 'b-', linewidth=1)
axes[0, 0].set_xlabel('Round')
axes[0, 0].set_ylabel('Gini Coefficient')
axes[0, 0].set_title('Wealth Inequality (Gini)')
axes[0, 0].axhline(y=0.4, color='darkorange', linestyle='--', alpha=0.5, label='Moderate inequality')
axes[0, 0].axhline(y=0.6, color='firebrick', linestyle='--', alpha=0.5, label='High inequality')
axes[0, 0].legend(fontsize=8)
axes[0, 0].grid(alpha=0.3)

# Active agents and participation
axes[0, 1].plot(health['round'], health['active_agents'], 'g-', linewidth=1.5)
axes[0, 1].set_xlabel('Round')
axes[0, 1].set_ylabel('Active Agents')
axes[0, 1].set_title('Active Participants')
axes[0, 1].grid(alpha=0.3)

# Collusion risk
axes[1, 0].plot(health['round'], health['collusion_risk'], 'r-', linewidth=1)
axes[1, 0].set_xlabel('Round')
axes[1, 0].set_ylabel('Collusion Risk')
axes[1, 0].set_title('Predatory Agent Fraction (Collusion Risk)')
axes[1, 0].grid(alpha=0.3)

# Pool health
axes[1, 1].plot(health['round'], health['pool_tvl'], 'purple', linewidth=1)
axes[1, 1].set_xlabel('Round')
axes[1, 1].set_ylabel('TVL')
axes[1, 1].set_title('AMM Pool TVL')
axes[1, 1].grid(alpha=0.3)

# Pool price
axes[2, 0].plot(health['round'], health['pool_price'], 'darkorange', linewidth=1)
axes[2, 0].set_xlabel('Round')
axes[2, 0].set_ylabel('Price (salt per iron)')
axes[2, 0].set_title('AMM Price Evolution')
axes[2, 0].grid(alpha=0.3)

# Token supply
axes[2, 1].plot(health['round'], health['total_supply'], 'steelblue', linewidth=1)
axes[2, 1].set_xlabel('Round')
axes[2, 1].set_ylabel('Total GLD Supply')
axes[2, 1].set_title('Token Supply (inflationary from staking rewards)')
axes[2, 1].grid(alpha=0.3)

plt.suptitle('Guild Economy Health Dashboard (1,000 Rounds)', fontsize=14)
plt.tight_layout()
plt.show()

In [ ]:
# Final system state
final_h = economy.health_history[-1]

print("=== FINAL SYSTEM STATE ===")
print(f"  Rounds completed: {N_ROUNDS}")
print(f"  Active agents: {final_h['active_agents']}/{len(agents)}")
print(f"  Ejected: {final_h['ejected']}")
print(f"  Gini coefficient: {final_h['gini']:.3f}")
print(f"  Collusion risk: {final_h['collusion_risk']:.2f}")
print(f"  Pool TVL: {final_h['pool_tvl']:,.0f}")
print(f"  Pool price: {final_h['pool_price']:.4f} salt/iron")
print(f"  Total GLD supply: {final_h['total_supply']:,.0f}")
print(f"  Total trades: {final_h['total_trades']}")
print(f"  Total fees collected: {economy.pool.total_fees:,.1f}")
print(f"  Total MEV events: {len(economy.mev_events)}")

## Part 6: Agent-Level Analysis

In [ ]:
# Build agent performance table
agent_data = []
for a in agents:
    ejected = economy.reputation.is_ejected(a.name)
    stake_info = economy.reputation.stakes.get(a.name, {'amount': 0, 'violations': 0})
    agent_data.append({
        'name': a.name,
        'strategy': a.strategy,
        'lockup': a.lockup,
        'final_gld': economy.token.balances[a.name],
        'stake': stake_info['amount'],
        'violations': stake_info['violations'],
        'rounds_active': a.rounds_active,
        'total_profit': a.total_profit,
        'mev_attempts': a.mev_attempts,
        'mev_profits': a.mev_profits,
        'times_slashed': a.times_slashed,
        'ejected': ejected
    })

agents_df = pd.DataFrame(agent_data).sort_values('final_gld', ascending=False)

print("Agent Performance Ranking:")
print(f"{'Name':20s} {'Strategy':13s} {'GLD':>10s} {'Stake':>8s} "
      f"{'Slash':>5s} {'MEV':>5s} {'Status':>8s}")
for _, row in agents_df.iterrows():
    status = 'EJECTED' if row['ejected'] else 'active'
    print(f"{row['name']:20s} {row['strategy']:13s} {row['final_gld']:10.0f} "
          f"{row['stake']:8.0f} {int(row['times_slashed']):5d} "
          f"{int(row['mev_attempts']):5d} {status:>8s}")

In [ ]:
# Strategy comparison
strategy_stats = agents_df.groupby('strategy').agg(
    count=('name', 'count'),
    avg_gld=('final_gld', 'mean'),
    median_gld=('final_gld', 'median'),
    avg_profit=('total_profit', 'mean'),
    ejection_rate=('ejected', 'mean'),
    avg_slashes=('times_slashed', 'mean'),
    avg_rounds=('rounds_active', 'mean')
)

print("Strategy Performance Comparison:")
print(f"{'Strategy':15s} {'Count':>5s} {'Avg GLD':>10s} {'Avg Profit':>11s} "
      f"{'Ejection%':>10s} {'Avg Rounds':>11s}")
for strategy, row in strategy_stats.iterrows():
    marker = ' ← BEST' if row['avg_gld'] == strategy_stats['avg_gld'].max() else ''
    print(f"{strategy:15s} {int(row['count']):5d} {row['avg_gld']:10.0f} "
          f"{row['avg_profit']:11.0f} {row['ejection_rate']:10.0%} "
          f"{row['avg_rounds']:11.0f}{marker}")

print(f"\nDoes honesty pay? "
      f"{'YES — honest agents have the highest average wealth.' if strategy_stats.loc['honest', 'avg_gld'] == strategy_stats['avg_gld'].max() else 'The results are more nuanced — see analysis below.'}")

In [ ]:
# Wealth distribution by strategy
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

colors = {'honest': 'steelblue', 'opportunistic': 'darkorange', 'predatory': 'firebrick'}

for strategy in ['honest', 'opportunistic', 'predatory']:
    subset = agents_df[agents_df['strategy'] == strategy]
    ax1.hist(subset['final_gld'], bins=10, alpha=0.6, color=colors[strategy],
             label=f'{strategy} (n={len(subset)})', edgecolor='black')

ax1.set_xlabel('Final GLD Balance')
ax1.set_ylabel('Count')
ax1.set_title('Wealth Distribution by Strategy')
ax1.legend()
ax1.grid(alpha=0.3)

# Box plot
strategy_groups = [agents_df[agents_df['strategy'] == s]['final_gld'].values 
                   for s in ['honest', 'opportunistic', 'predatory']]
bp = ax2.boxplot(strategy_groups, labels=['honest', 'opportunistic', 'predatory'],
                 patch_artist=True)
for patch, color in zip(bp['boxes'], [colors['honest'], colors['opportunistic'], colors['predatory']]):
    patch.set_facecolor(color)
    patch.set_alpha(0.6)
ax2.set_ylabel('Final GLD Balance')
ax2.set_title('Wealth by Strategy (Box Plot)')
ax2.grid(alpha=0.3)

plt.tight_layout()
plt.show()

## Part 7: Cross-Dataset Validation

Let's validate our simulation results against the pre-generated datasets.

In [ ]:
# Compare simulation patterns to historical datasets
print("=== CROSS-DATASET VALIDATION ===")
print()

# 1. Fraud rate comparison
events_df = pd.DataFrame(economy.history)
if len(events_df) > 0:
    fraud_events = events_df[events_df['action'].isin(['fraud_detected', 'fraud_undetected'])]
    detected = (fraud_events['action'] == 'fraud_detected').sum()
    total_fraud = len(fraud_events)
    sim_detection_rate = detected / max(total_fraud, 1)
    
    hist_fraud_rate = guild_trades['fraud_detected'].mean()
    
    print(f"1. Fraud Detection:")
    print(f"   Historical (guild_trade_ledger.csv): {hist_fraud_rate:.1%} detected")
    print(f"   Simulation: {sim_detection_rate:.1%} detected")
    print(f"   Staking increases detection through aligned incentives.")

# 2. Ejection patterns
hist_ejected = staking[staking['status'] == 'ejected']['staker'].nunique()
hist_total = staking['staker'].nunique()
sim_ejected = sum(1 for a in agents if economy.reputation.is_ejected(a.name))

print(f"\n2. Ejection Rate:")
print(f"   Historical (staking_rewards.csv): {hist_ejected}/{hist_total} ({hist_ejected/hist_total:.0%})")
print(f"   Simulation: {sim_ejected}/{len(agents)} ({sim_ejected/len(agents):.0%})")

# 3. MEV extraction
hist_mev_profit = mev_log['net_profit'].sum()
sim_mev_profit = sum(e['profit'] for e in economy.mev_events)

print(f"\n3. MEV Extraction:")
print(f"   Historical (mev_extraction_log.csv): {hist_mev_profit:,.0f} total net profit")
print(f"   Simulation: {sim_mev_profit:,.0f} total net profit")

# 4. Strategy performance vs game theory predictions
final_gt = game_theory[game_theory['round'] == 10]
gt_best = final_gt.groupby('strategy_a')['cumulative_payoff_a'].mean().idxmax()

print(f"\n4. Strategy Validation:")
print(f"   Game theory tournament winner: {gt_best}")
best_sim = strategy_stats['avg_gld'].idxmax()
print(f"   Economy simulation winner: {best_sim}")
print(f"   Consistent with mechanism design theory: cooperation dominates in staked systems.")

## Part 8: The Complete Auster Arc

Let's trace the full narrative arc from problem to solution.

In [ ]:
# The Auster arc: problem → theory → design → implementation → measurement
print("=" * 70)
print("THE AUSTER ARC: From Enforcement Failure to Self-Enforcing Economy")
print("=" * 70)

# Phase 1: The Problem (Tutorial 01)
dispute_rate = (guild_trades['outcome'] != 'completed').mean()
bribe_rate = (guild_trades['inspector_ruling'] == 'inspector_bribed').sum() / len(guild_trades)
print(f"\n1. THE PROBLEM (Tutorial 01):")
print(f"   Guild dispute rate: {dispute_rate:.1%}")
print(f"   Inspector bribery: {bribe_rate:.1%} of all trades")
print(f"   → Enforcement-based systems fail when enforcers defect.")

# Phase 2: The Theory (Tutorials 02-03)
final_rounds = game_theory[game_theory['round'] == 10]
tft_score = final_rounds[final_rounds['strategy_a'] == 'tit_for_tat']['cumulative_payoff_a'].mean()
defect_score = final_rounds[final_rounds['strategy_a'] == 'always_defect']['cumulative_payoff_a'].mean()
print(f"\n2. THE THEORY (Tutorials 02-03):")
print(f"   Tit-for-tat avg payoff: {tft_score:.1f}")
print(f"   Always-defect avg payoff: {defect_score:.1f}")
print(f"   → Cooperation dominates in repeated games (Folk Theorem).")
print(f"   → Mechanism design: engineer games where honesty is dominant.")

# Phase 3: The Implementation (Tutorials 04-07)
print(f"\n3. THE IMPLEMENTATION (Tutorials 04-07):")
print(f"   Token system: {economy.token.total_supply:,.0f} GLD in circulation")
active_stakes = sum(1 for s in economy.reputation.stakes.values() if s['amount'] > 0)
print(f"   Staking: {active_stakes} active stakers")
print(f"   AMM: {economy.pool.trade_count} trades, {economy.pool.total_fees:,.0f} fees")
print(f"   MEV: {len(economy.mev_events)} extraction events")

# Phase 4: The Result
print(f"\n4. THE RESULT (This Capstone):")
honest_avg = agents_df[agents_df['strategy'] == 'honest']['final_gld'].mean()
predatory_avg = agents_df[agents_df['strategy'] == 'predatory']['final_gld'].mean()
print(f"   Honest agent avg wealth: {honest_avg:,.0f} GLD")
print(f"   Predatory agent avg wealth: {predatory_avg:,.0f} GLD")
print(f"   Wealth ratio (honest/predatory): {honest_avg/max(predatory_avg, 1):.1f}x")
print(f"   Predatory ejection rate: {agents_df[agents_df['strategy'] == 'predatory']['ejected'].mean():.0%}")
print(f"   → The system self-enforces. Honesty is the cheapest strategy.")

print(f"\n" + "=" * 70)
print(f"Auster was right: you don't need judges. You need mathematics.")
print(f"=" * 70)

## Exercises

### Exercise 1: Parameter Sensitivity

How sensitive is the "honesty dominates" result to the slash rate? Run the simulation with slash rates of 1%, 5%, 10%, and 20% and compare strategy outcomes.

In [ ]:
# Parameter sensitivity: vary slash rate
slash_rates = [0.01, 0.05, 0.10, 0.20]
sensitivity_results = []

for sr in slash_rates:
    np.random.seed(867)
    test_agents = []
    test_strategies = ['honest'] * 18 + ['opportunistic'] * 7 + ['predatory'] * 5
    np.random.shuffle(test_strategies)
    for name, strategy in zip(agent_names, test_strategies):
        test_agents.append(GuildAgent(name, strategy, np.random.uniform(500, 2000),
                                      np.random.choice([1, 2, 4, 8], p=[0.3, 0.3, 0.25, 0.15])))
    
    test_econ = GuildEconomy(test_agents)
    test_econ.reputation.SLASH_RATE = sr
    
    for _ in range(500):  # Shorter run for speed
        test_econ.run_round()
    
    for a in test_agents:
        sensitivity_results.append({
            'slash_rate': sr,
            'strategy': a.strategy,
            'final_gld': test_econ.token.balances[a.name],
            'ejected': test_econ.reputation.is_ejected(a.name)
        })

sens_df = pd.DataFrame(sensitivity_results)

# Plot
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

for strategy in ['honest', 'opportunistic', 'predatory']:
    subset = sens_df[sens_df['strategy'] == strategy]
    avg_by_sr = subset.groupby('slash_rate')['final_gld'].mean()
    color = {'honest': 'steelblue', 'opportunistic': 'darkorange', 'predatory': 'firebrick'}[strategy]
    ax1.plot(avg_by_sr.index * 100, avg_by_sr.values, 'o-', color=color,
             linewidth=2, label=strategy, markersize=8)

ax1.set_xlabel('Slash Rate %')
ax1.set_ylabel('Average Final GLD')
ax1.set_title('Strategy Returns vs Slash Rate')
ax1.legend()
ax1.grid(alpha=0.3)

# Ejection rate
for strategy in ['honest', 'opportunistic', 'predatory']:
    subset = sens_df[sens_df['strategy'] == strategy]
    eject_by_sr = subset.groupby('slash_rate')['ejected'].mean()
    color = {'honest': 'steelblue', 'opportunistic': 'darkorange', 'predatory': 'firebrick'}[strategy]
    ax2.plot(eject_by_sr.index * 100, eject_by_sr.values * 100, 'o-', color=color,
             linewidth=2, label=strategy, markersize=8)

ax2.set_xlabel('Slash Rate %')
ax2.set_ylabel('Ejection Rate %')
ax2.set_title('Ejection Rate vs Slash Rate')
ax2.legend()
ax2.grid(alpha=0.3)

plt.tight_layout()
plt.show()

print("Higher slash rates punish predatory agents more severely.")
print("But too high and opportunistic agents also get ejected, reducing participation.")

### Exercise 2: Strategy Population Dynamics

What happens if predatory agents are the majority? Run with 50% predatory and see if the system survives.

In [ ]:
# Adversarial scenario: 50% predatory
np.random.seed(42)
adv_agents = []
adv_strategies = ['honest'] * 10 + ['opportunistic'] * 5 + ['predatory'] * 15
np.random.shuffle(adv_strategies)

for name, strategy in zip(agent_names, adv_strategies):
    adv_agents.append(GuildAgent(name, strategy, np.random.uniform(500, 2000),
                                  np.random.choice([1, 2, 4, 8], p=[0.3, 0.3, 0.25, 0.15])))

adv_econ = GuildEconomy(adv_agents)
for _ in range(500):
    adv_econ.run_round()

adv_health = pd.DataFrame(adv_econ.health_history)

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

ax1.plot(adv_health['round'], adv_health['active_agents'], 'r-', linewidth=1.5, label='50% predatory')
ax1.plot(health['round'][:500], health['active_agents'][:500], 'b-', linewidth=1.5, label='15% predatory')
ax1.set_xlabel('Round')
ax1.set_ylabel('Active Agents')
ax1.set_title('Participation: Normal vs Adversarial')
ax1.legend()
ax1.grid(alpha=0.3)

ax2.plot(adv_health['round'], adv_health['collusion_risk'], 'r-', linewidth=1.5, label='50% predatory')
ax2.plot(health['round'][:500], health['collusion_risk'][:500], 'b-', linewidth=1.5, label='15% predatory')
ax2.set_xlabel('Round')
ax2.set_ylabel('Collusion Risk')
ax2.set_title('Collusion Risk: Normal vs Adversarial')
ax2.legend()
ax2.grid(alpha=0.3)

plt.tight_layout()
plt.show()

final_adv = adv_econ.health_history[-1]
print(f"50% predatory scenario after 500 rounds:")
print(f"  Active agents: {final_adv['active_agents']}/{len(adv_agents)}")
print(f"  Collusion risk: {final_adv['collusion_risk']:.2f}")
print(f"  The system ejects predatory agents faster when there are more of them.")
print(f"  Staking + slashing is resilient even under adversarial conditions.")

### Exercise 3: System Health Score

Design a composite health score (0-100) that combines Gini, participation, collusion risk, and TVL stability into a single metric.

In [ ]:
def compute_health_score(health_row):
    """Composite health score (0-100). Higher is healthier."""
    # Equality (low Gini is good)
    equality_score = max(0, (1 - health_row['gini']) * 25)
    
    # Participation (high is good)
    participation_score = health_row['participation_rate'] * 25
    
    # Safety (low collusion risk is good)
    safety_score = max(0, (1 - health_row['collusion_risk'] * 2)) * 25
    
    # Liquidity (stable TVL is good)
    liquidity_score = min(health_row['pool_tvl'] / 5000, 1.0) * 25
    
    return equality_score + participation_score + safety_score + liquidity_score

health['health_score'] = health.apply(compute_health_score, axis=1)

plt.figure(figsize=(12, 5))
plt.plot(health['round'], health['health_score'], 'steelblue', linewidth=1.5)
plt.axhline(y=75, color='forestgreen', linestyle='--', alpha=0.5, label='Healthy (>75)')
plt.axhline(y=50, color='darkorange', linestyle='--', alpha=0.5, label='Warning (50-75)')
plt.axhline(y=25, color='firebrick', linestyle='--', alpha=0.5, label='Critical (<25)')
plt.xlabel('Round')
plt.ylabel('Health Score (0-100)')
plt.title('Composite System Health Score')
plt.legend()
plt.grid(alpha=0.3)
plt.ylim(0, 100)
plt.tight_layout()
plt.show()

final_score = health['health_score'].iloc[-1]
avg_score = health['health_score'].mean()
print(f"Final health score: {final_score:.1f}/100")
print(f"Average health score: {avg_score:.1f}/100")
print(f"Min health score: {health['health_score'].min():.1f}/100 (round {health['health_score'].idxmin()})")

if final_score > 75:
    print(f"\nSystem is HEALTHY. Auster's mechanism design works.")
elif final_score > 50:
    print(f"\nSystem is STABLE but showing stress. Parameters may need adjustment.")
else:
    print(f"\nSystem is STRUGGLING. The incentive structure needs redesign.")

## Summary

In this capstone, we built and ran a complete guild economy:

1. **30 agents** with three strategies (honest, opportunistic, predatory) running for **1,000 rounds**
2. **GLD tokens** as programmable currency with staked reputation
3. **AMM trading** for trustless commodity exchange with fee distribution
4. **MEV extraction** as a real-world risk to the system
5. **System health metrics**: Gini inequality, participation rates, collusion risk, composite health score

**The central result:** Honest agents accumulate more wealth than predatory agents. The staking mechanism makes honesty the dominant strategy — not through rules or enforcement, but through mathematical incentive design. Predatory agents get slashed, ejected, and outperformed.

This validates Auster's thesis from Tutorial 01: *"Do not ask whether the guild member will cheat. Ask what it will cost them."*

### The Full Course Arc

| Tutorial | Question | Answer |
|----------|----------|--------|
| 01 | Why do traders cheat? | Because the Prisoner's Dilemma makes defection rational |
| 02 | When can cooperation survive? | In repeated games, tit-for-tat dominates |
| 03 | Can we engineer cooperation? | Yes — mechanism design makes honesty dominant |
| 04 | What are the building blocks? | ERC-20 and ERC-721 tokens as programmable state |
| 05 | How do we enforce it? | Staking and slashing — post collateral, lose it for cheating |
| 06 | How do we trade? | AMMs — constant product formula, trustless exchange |
| 07 | What can go wrong? | MEV — the observer effect of transparent mempools |
| 08 | Does it all work together? | **Yes. Honesty is the cheapest strategy.** |

---

> *"The Capital tried rules. The Guild tried honor. Both failed. I tried mathematics — and discovered that honesty is just the cheapest strategy."*
>
> — Brenn Auster, *Self-Enforcing Agreements* (Year 867)